In [1]:
import os

# Set the environment variables before using the transformers library
# os.environ["HF_HOME"] = "/serenity/scratch/hkolisetty6/.cache/huggingface"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import sys
sys.path.insert(0, "./transformers/src")
sys.path.insert(0, "./peft/src")

In [2]:
from profiler import (
    parse_args,
    get_last_checkpoint,
    load_model,
    set_width_mask_and_bias,
    make_data_module,
    Seq2SeqTrainer,
    profile_latencies,
    get_latency_stats,
    logging,
)
import json

/home/dsanyal7/miniconda3/envs/amoeballm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
args = [
    "--model_name_or_path", "meta-llama/Llama-2-7b-hf",
    "--output_dir", "amoeba_llama2",
    "--do_predict", "True",
    "--do_eval", "False",
    "--do_train", "False",
    "--do_mmlu_eval", "False",
    "--enable_shrinking",
    "--min_num_layer", "20",
    "--shrinking_method", "calib_dp",
    "--shrinking_file", "dp_selection_strategy.npy",
    "--shrinkable_width",
    "--width_choice", "[1,7/8,3/4,5/8,1/2]",
    "--prune_width_method", "flap",
    "--use_moe_lora",
    "--moe_num_expert", "5",
    "--moe_topk", "2",
    "--eval_num_layer", "32",
    "--eval_num_width", "1",
    "--predict_with_generate", "True",
    "--source_max_len", "512",
    "--max_new_tokens", "32", # TODO: hkolisetty6
    # "--bf16",
]
args, training_args = parse_args(args_list=args)
checkpoint_dir = get_last_checkpoint(args.output_dir)
model, tokenizer = load_model(args, checkpoint_dir)
set_width_mask_and_bias(model, args)
logger = logging.getLogger(__name__)
results = {}

Using compute dtype: torch.float32


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.47s/it]


Adding correct special tokens to the llama tokenizer
Loading adapters from checkpoint


In [6]:
eval_num_layers = range(16, 33)
for num_layers in eval_num_layers:
    print(f"Evaluating with num_layers: {num_layers}")

    args.eval_num_layer = num_layers
    key = f'l{args.eval_num_layer}w{args.eval_num_width}sml{args.source_max_len}mnt32'

    results[key] = []
    ttfts = []
    tbts = []
    for bs in [1, 2, 4]:
        training_args.per_device_eval_batch_size = bs
        args.eval_dataset_size = training_args.per_device_eval_batch_size * 70 + 1
        data_module = make_data_module(tokenizer=tokenizer, args=args)
        trainer = Seq2SeqTrainer(
            model=model,
            tokenizer=tokenizer,
            args=training_args,
            **{k: v for k, v in data_module.items() if k != "predict_dataset"},
        )
        ttft, tbt = profile_latencies(model, tokenizer, args, logger, trainer, data_module)
        ttfts.append(ttft)
        tbts.append(tbt)
        results[key].append(get_latency_stats(ttft, tbt, bs))
    
    with open(f'latency_results_{key}.json', 'w') as fout:
        json.dump(results, fout)
    with open(f'latencies_{key}.json', 'w') as fout:
        json.dump({'ttft': ttfts, 'tbt': tbts}, fout)
    

Evaluating with num_layers: 16
Splitting train dataset in train and validation according to `eval_dataset_size`


{1: (1, 0.2154464554041624), 2: (1, 0.19507850240916014), 3: (1, 0.1941644735634327), 4: (1, 0.19450733251869678), 5: (1, 0.1942154634743929), 6: (1, 0.19414843432605267), 7: (1, 0.19376584608107805), 8: (1, 0.19385494571179152), 9: (1, 0.1935255266726017), 10: (1, 0.19397968333214521), 11: (1, 0.1936962753534317), 12: (1, 0.19496701192110777), 13: (1, 0.19417630415409803), 14: (1, 0.19409166555851698), 15: (1, 0.19429887365549803), 16: (1, 0.1945168236270547), 17: (1, 0.194290554150939), 18: (1, 0.19475911278277636), 19: (1, 0.1944847833365202), 20: (1, 0.19499493204057217), 21: (1, 0.19442474283277988), 22: (1, 0.19405509438365698), 23: (1, 0.19418378453701735), 24: (1, 0.1960844285786152), 25: (1, 0.19446810334920883), 26: (1, 0.1944258138537407), 27: (1, 0.1942357448861003), 28: (1, 0.19435006380081177), 29: (1, 0.1943665537983179), 30: (1, 0.1943148747086525), 31: (1, 0.1941770650446415), 32: (1, 0.19398080557584763), 33: (1, 0.19463392347097397), 34: (1, 0.19416782446205616), 35:

{1: (2, 0.32418809924274683), 2: (2, 0.30336253717541695), 3: (2, 0.3025214998051524), 4: (2, 0.30370423570275307), 5: (2, 0.3027303786948323), 6: (2, 0.3036179067566991), 7: (2, 0.3030100194737315), 8: (2, 0.3030607383698225), 9: (2, 0.3037814563140273), 10: (2, 0.3030804777517915), 11: (2, 0.3046946134418249), 12: (2, 0.3022612202912569), 13: (2, 0.3035259572789073), 14: (2, 0.3032721569761634), 15: (2, 0.3023851700127125), 16: (2, 0.30359999649226665), 17: (2, 0.30303522758185863), 18: (2, 0.30279869865626097), 19: (2, 0.3035506969317794), 20: (2, 0.303690736182034), 21: (2, 0.30394260585308075), 22: (2, 0.3032279182225466), 23: (2, 0.30301069747656584), 24: (2, 0.3033811869099736), 25: (2, 0.30390753597021103), 26: (2, 0.30328442715108395), 27: (2, 0.30310252867639065), 28: (2, 0.3035471374168992), 29: (2, 0.30360604729503393), 30: (2, 0.30369577649980783), 31: (2, 0.30330115742981434), 32: (2, 0.3037834856659174), 33: (2, 0.3033529482781887), 34: (2, 0.30342039838433266), 35: (2, 

{1: (4, 0.5306851174682379), 2: (4, 0.5118727283552289), 3: (4, 0.5126386154443026), 4: (4, 0.5119033986702561), 5: (4, 0.5118636693805456), 6: (4, 0.5135332746431231), 7: (4, 0.5125452568754554), 8: (4, 0.5127576272934675), 9: (4, 0.5130177959799767), 10: (4, 0.5137245925143361), 11: (4, 0.5132101662456989), 12: (4, 0.5136367734521627), 13: (4, 0.5131777748465538), 14: (4, 0.5129288760945201), 15: (4, 0.5129942065104842), 16: (4, 0.5131893744692206), 17: (4, 0.5127526363357902), 18: (4, 0.5124373380094767), 19: (4, 0.514436331577599), 20: (4, 0.512881807051599), 21: (4, 0.512678206898272), 22: (4, 0.5129269864410162), 23: (4, 0.5139569826424122), 24: (4, 0.5122425565496087), 25: (4, 0.5127257956191897), 26: (4, 0.5129443863406777), 27: (4, 0.5128410160541534), 28: (4, 0.5135106649249792), 29: (4, 0.5123666366562247), 30: (4, 0.5126238064840436), 31: (4, 0.5131109161302447), 32: (4, 0.5125136664137244), 33: (4, 0.5129943443462253), 34: (4, 0.5125618865713477), 35: (4, 0.512571967206895

{1: (1, 0.22555291559547186), 2: (1, 0.2052050605416298), 3: (1, 0.20562755968421698), 4: (1, 0.20776396431028843), 5: (1, 0.2054717792198062), 6: (1, 0.20575285889208317), 7: (1, 0.2054868098348379), 8: (1, 0.2055043699219823), 9: (1, 0.20524345058947802), 10: (1, 0.20580639876425266), 11: (1, 0.2053157603368163), 12: (1, 0.20546347927302122), 13: (1, 0.20535563956946135), 14: (1, 0.20556671917438507), 15: (1, 0.20535598043352365), 16: (1, 0.20545264892280102), 17: (1, 0.20526948012411594), 18: (1, 0.2055917987599969), 19: (1, 0.20635422691702843), 20: (1, 0.20599001832306385), 21: (1, 0.2056388696655631), 22: (1, 0.20601198822259903), 23: (1, 0.20617028791457415), 24: (1, 0.20599541906267405), 25: (1, 0.20573408901691437), 26: (1, 0.2061774069443345), 27: (1, 0.2065860154107213), 28: (1, 0.20607763808220625), 29: (1, 0.20623945724219084), 30: (1, 0.20588107965886593), 31: (1, 0.2079560523852706), 32: (1, 0.2061814172193408), 33: (1, 0.20586353726685047), 34: (1, 0.20577167812734842),

...

{1: (2, 0.5996599039062858), 2: (2, 0.5811790572479367), 3: (2, 0.5811515785753727), 4: (2, 0.5794722633436322), 5: (2, 0.5788492048159242), 6: (2, 0.5789784947410226), 7: (2, 0.5799163114279509), 8: (2, 0.5801028115674853), 9: (2, 0.5818259362131357), 10: (2, 0.5799478013068438), 11: (2, 0.5806315038353205), 12: (2, 0.5808225441724062), 13: (2, 0.581814200617373), 14: (2, 0.5806799931451678), 15: (2, 0.5800679046660662), 16: (2, 0.5803181743249297), 17: (2, 0.5807069623842835), 18: (2, 0.5815231408923864), 19: (2, 0.581597151234746), 20: (2, 0.5806483738124371), 21: (2, 0.580376984551549), 22: (2, 0.5793551579117775), 23: (2, 0.5804408146068454), 24: (2, 0.581183422356844), 25: (2, 0.5803900454193354), 26: (2, 0.5803867839276791), 27: (2, 0.5813779318705201), 28: (2, 0.5801320727914572), 29: (2, 0.5811942284926772), 30: (2, 0.5802686717361212), 31: (2, 0.5802784003317356), 32: (2, 0.5808845702558756), 33: (2, 0.5809160182252526), 34: (2, 0.5816778978332877), 35: (2, 0.5804875306785107

{1: (4, 0.99675886426121), 2: (4, 0.9752134261652827), 3: (4, 0.973984070122242), 4: (4, 0.9766279524192214), 5: (4, 0.9746818281710148), 6: (4, 0.9749543378129601), 7: (4, 0.9750739466398954), 8: (4, 0.9744156897068024), 9: (4, 0.9748884281143546), 10: (4, 0.9755342258140445), 11: (4, 0.9765941528603435), 12: (4, 0.9767459826543927), 13: (4, 0.976832932792604), 14: (4, 0.9768917923793197), 15: (4, 0.9767590221017599), 16: (4, 0.9769957214593887), 17: (4, 0.9767021620646119), 18: (4, 0.9764538230374455), 19: (4, 0.9770257119089365), 20: (4, 0.9764167340472341), 21: (4, 0.9765824330970645), 22: (4, 0.9772541960701346), 23: (4, 0.9766531884670258), 24: (4, 0.977681215852499), 25: (4, 0.9770033974200487), 26: (4, 0.9770102081820369), 27: (4, 0.9771186681464314), 28: (4, 0.9775892067700624), 29: (4, 0.9772836267948151), 30: (4, 0.9767999080941081), 31: (4, 0.9771846868097782), 32: (4, 0.9773846762254834), 33: (4, 0.9763461090624332), 34: (4, 0.9770416971296072), 35: (4, 0.9767049383372068)

{1: (1, 0.40125147346407175), 2: (1, 0.3812619736418128), 3: (1, 0.38040116522461176), 4: (1, 0.3803508058190346), 5: (1, 0.3798735374584794), 6: (1, 0.38022057618945837), 7: (1, 0.380195795558393), 8: (1, 0.3801930360496044), 9: (1, 0.37962453812360764), 10: (1, 0.38126498460769653), 11: (1, 0.38023470621556044), 12: (1, 0.3801853861659765), 13: (1, 0.38014693558216095), 14: (1, 0.38082268461585045), 15: (1, 0.3804576760157943), 16: (1, 0.3803910156711936), 17: (1, 0.38012788631021976), 18: (1, 0.3800801169127226), 19: (1, 0.38005672581493855), 20: (1, 0.3807776318863034), 21: (1, 0.3796946741640568), 22: (1, 0.379853623919189), 23: (1, 0.3800199432298541), 24: (1, 0.38092026114463806), 25: (1, 0.3796419743448496), 26: (1, 0.38048364222049713), 27: (1, 0.37978786416351795), 28: (1, 0.3801573533564806), 29: (1, 0.3799483338370919), 30: (1, 0.38007729314267635), 31: (1, 0.3807773608714342), 32: (1, 0.38014907389879227), 33: (1, 0.37987974379211664), 34: (1, 0.3801329731941223), 35: (1, 

{1: (2, 0.6188558172434568), 2: (2, 0.5974686425179243), 3: (2, 0.5966535061597824), 4: (2, 0.5950609501451254), 5: (2, 0.5968484356999397), 6: (2, 0.59613027703017), 7: (2, 0.5960262268781662), 8: (2, 0.5955394888296723), 9: (2, 0.5955837778747082), 10: (2, 0.5964759457856417), 11: (2, 0.5964269461110234), 12: (2, 0.5961735565215349), 13: (2, 0.596158217638731), 14: (2, 0.5965920751914382), 15: (2, 0.5960332565009594), 16: (2, 0.5968562746420503), 17: (2, 0.596421635709703), 18: (2, 0.5961192697286606), 19: (2, 0.5959740309044719), 20: (2, 0.5962624093517661), 21: (2, 0.5963249392807484), 22: (2, 0.5967129375785589), 23: (2, 0.5960973193868995), 24: (2, 0.596634947694838), 25: (2, 0.5965418079867959), 26: (2, 0.5970078380778432), 27: (2, 0.5959938308224082), 28: (2, 0.596183049492538), 29: (2, 0.5967862578108907), 30: (2, 0.5961213689297438), 31: (2, 0.5957877216860652), 32: (2, 0.5965799074620008), 33: (2, 0.597029966302216), 34: (2, 0.5957999508827925), 35: (2, 0.5956223113462329), 

{1: (4, 1.0256664454936981), 2: (4, 1.0047647841274738), 3: (4, 1.006563389673829), 4: (4, 1.0057952012866735), 5: (4, 1.0060192300006747), 6: (4, 1.0059685604646802), 7: (4, 1.0066113788634539), 8: (4, 1.0075784660875797), 9: (4, 1.0063643101602793), 10: (4, 1.0065463203936815), 11: (4, 1.0087432879954576), 12: (4, 1.0081871999427676), 13: (4, 1.009379635564983), 14: (4, 1.0111064007505774), 15: (4, 1.0091453567147255), 16: (4, 1.0090582873672247), 17: (4, 1.0095064053311944), 18: (4, 1.0082912994548678), 19: (4, 1.0094763459637761), 20: (4, 1.008516508154571), 21: (4, 1.008849618025124), 22: (4, 1.0082094594836235), 23: (4, 1.0084519693627954), 24: (4, 1.0081815514713526), 25: (4, 1.0085828676819801), 26: (4, 1.0084751984104514), 27: (4, 1.0076411599293351), 28: (4, 1.008320857770741), 29: (4, 1.009606333449483), 30: (4, 1.0088317655026913), 31: (4, 1.0090158265084028), 32: (4, 1.0078699104487896), 33: (4, 1.0093762846663594), 34: (4, 1.00908147636801), 35: (4, 1.0082770185545087), 3